In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
import numpy as np
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
import pandas as pd
import cufflinks as cf
from django.db.models import Q
from apps.environment_simulator.sevice_layer.market_simulator import MarketSimulator
from apps.environment_simulator.models import SimulatedStockBuffer
import gymnasium as gym
import ray
from gymnasium import spaces
from ray.rllib.algorithms import ppo
from os import path, environ

2023-10-07 22:03:08,044	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}

config = {
        "env": MarketSimulator,
        "env_config": {"db_params": db_params},
        "num_workers": 4,
}

ray.shutdown()
ray.init(runtime_env={"working_dir": "/Users/mirbilal/Desktop/MobCommission/commissionV2/"})
algo = ppo.PPO(config=config)

2023-10-07 22:03:11,324	INFO worker.py:1642 -- Started a local Ray instance.
2023-10-07 22:03:11,666	INFO packaging.py:518 -- Creating a file package for local directory '/Users/mirbilal/Desktop/MobCommission/commissionV2/'.
2023-10-07 22:03:11,921	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_f6eaca355ab3652a.zip' (6.56MiB) to Ray cluster...
2023-10-07 22:03:11,951	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_f6eaca355ab3652a.zip'.
2023-10-07 22:03:13,478	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-pack

(RolloutWorker pid=18361) {'db_params': {'database': 'traderdb', 'user': 'traderadmin', 'password': 'i4n6s9l1y', 'host': 'localhost', 'port': '5432'}, worker=4/4, vector_idx=0, remote=False}


(RolloutWorker pid=18360) /Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/gymnasium/wrappers/compatibility.py:67: DeprecationWarning: WARN: The `gymnasium.make(..., apply_api_compatibility=...)` parameter is deprecated and will be removed in v0.29. Instead use `gym.make('GymV21Environment-v0', env_name=...)` or `from shimmy import GymV21CompatibilityV0`
(RolloutWorker pid=18360)   logger.deprecation(
(RolloutWorker pid=18360) 2023-10-07 22:03:23,083	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(RolloutWorker pid=18358) 2023-10-07 22:03:23,052	WARNING utils.py:152 -- `config.auto_wrap_old

In [4]:
ray.shutdown()

In [3]:
mean_ppo = []
for _ in range(50):

    result = algo.train()
    print("episode reward mean:", _, result['episode_reward_mean'])
    mean_ppo.append(result['episode_reward_mean'])


OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


episode reward mean: 0 -1990.0
episode reward mean: 1 -1990.0
episode reward mean: 2 -1990.0
episode reward mean: 3 -1990.0
episode reward mean: 4 -1990.0
episode reward mean: 5 -1990.0
episode reward mean: 6 -1990.0
episode reward mean: 7 -1990.0
episode reward mean: 8 -1990.0
episode reward mean: 9 -1990.0
episode reward mean: 10 -1990.0
episode reward mean: 11 -1990.0
episode reward mean: 12 -1990.0
episode reward mean: 13 -1990.0
episode reward mean: 14 -1990.0
episode reward mean: 15 -1990.0
episode reward mean: 16 -1990.0
episode reward mean: 17 -1990.0
episode reward mean: 18 -1990.0
episode reward mean: 19 -1990.0
episode reward mean: 20 -1990.0
episode reward mean: 21 -1990.0
episode reward mean: 22 -1990.0
episode reward mean: 23 -1990.0
episode reward mean: 24 -1990.0
episode reward mean: 25 -1990.0
episode reward mean: 26 -1990.0
episode reward mean: 27 -1990.0
episode reward mean: 28 -1990.0
episode reward mean: 29 -1990.0
episode reward mean: 30 -1990.0
episode reward mea

OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


episode reward mean: 43 -1990.0
episode reward mean: 44 -1990.0
episode reward mean: 45 -1990.0
episode reward mean: 46 -1990.0
episode reward mean: 47 -1990.0
episode reward mean: 48 -1990.0
episode reward mean: 49 -1990.0


In [15]:

new_env = MarketSimulator(env_config={"db_params": db_params})
state = new_env.state
# print(algo.workers.foreach_worker())
for remote_worker in algo.workers.remote_workers():
    remote_worker_state = remote_worker.get_state.remote()
    # print(f"Remote Worker State (Worker ID: {remote_worker.worker_index}):")
    print(ray.get(remote_worker_state))
    break

{'db_params': {'database': 'traderdb', 'user': 'traderadmin', 'password': 'i4n6s9l1y', 'host': 'localhost', 'port': '5432'}}
{'policy_ids': ['default_policy'], 'policy_states': {'default_policy': {'weights': {'encoder.actor_encoder.net.mlp.0.weight': array([[-0.13278131,  0.13457751,  0.12638482, ...,  0.07195501,
        -0.02048276,  0.06492655],
       [-0.1108738 ,  0.06767422,  0.02139008, ..., -0.12029484,
         0.08268021, -0.05853697],
       [ 0.09809622, -0.02476887,  0.02444181, ..., -0.05361897,
         0.04576394, -0.00744812],
       ...,
       [ 0.07241239,  0.00572939, -0.10221986, ..., -0.07793653,
        -0.13818647,  0.03838398],
       [ 0.06367998,  0.08289972, -0.02113632, ...,  0.08371337,
        -0.02836435,  0.06430763],
       [ 0.01065406, -0.00919558,  0.08874711, ...,  0.03057945,
        -0.12073731, -0.04604364]], dtype=float32), 'encoder.actor_encoder.net.mlp.0.bias': array([ 5.64876199e-03,  1.27166986e-01,  1.01224914e-01, -5.14570996e-02,
     

In [5]:
from gymnasium import spaces

no_of_stocks = 5
stock_observation_shape = (no_of_stocks, 8)

stock_lower_bounds = np.zeros(stock_observation_shape)
stock_upper_bounds = np.full(stock_observation_shape, float('inf'))

no_of_cmmdts = 6
commodity_observation_shape = (no_of_cmmdts, 1) 

commodity_lower_bounds = np.zeros(commodity_observation_shape)
commodity_upper_bounds = np.full(commodity_observation_shape, float('inf'))

stock_observation_space = spaces.Box(low=stock_lower_bounds, high=stock_upper_bounds)
commodity_observation_space = spaces.Box(low=commodity_lower_bounds, high=commodity_upper_bounds)

wallet_observation_space = spaces.Box(low=0, high=float('inf'), shape=(1,))
states = spaces.Tuple((stock_observation_space, commodity_observation_space, wallet_observation_space))

print(states)

Tuple(Box(0.0, inf, (5, 8), float32), Box(0.0, inf, (6, 1), float32), Box(0.0, inf, (1,), float32))


In [4]:
__time_step  = datetime(year=1995, month=1, day=1, hour=10)
__time_step = pytz.utc.localize(datetime.strptime(str(__time_step), '%Y-%m-%d %H:%M:%S'))
print(str(__time_step))

1995-01-01 10:00:00+00:00
